# Random Forest Bellwether Classifier

## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.svm import SVC
from sklearn import model_selection
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import BaggingRegressor
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import statistics

# check version number
import imblearn
print(imblearn.__version__)

0.8.0


## Load Dataset

In [3]:
data = pd.read_csv("data/segmented_data/data_2004_2020.csv")
data = data.drop(['Unnamed: 0'], axis=1)

In [4]:
data.head()

fips    state     county  wins  poverty_rate    population  \
0  1005  Alabama    Barbour     3     35.833333  2.458100e+04   
1  1035  Alabama    Conecuh     3     37.000000  1.507550e+04   
2  1073  Alabama  Jefferson     5     18.333333  6.437887e+05   
3  1091  Alabama    Marengo     4     36.500000  2.371617e+04   
4  4013  Arizona   Maricopa     4     13.166667  1.999060e+06   

   persons_in_poverty  rural_urban_code  urban_influence_code  metro  ...  \
0         8902.166667               6.0                   6.0    0.0  ...   
1         5833.833333               7.0                  11.0    0.0  ...   
2       120617.000000               1.0                   1.0    1.0  ...   
3         9067.000000               7.0                  11.0    0.0  ...   
4       269724.333333               1.0                   1.0    1.0  ...   

   percent_black_male  percent_black_female  percent_native_american_male  \
0           25.180116             22.500706                      0.377976   
1           21.227098             25.178039                      0.243664   
2           19.539034             23.318114                      0.164690   
3           23.493891             28.237603                      0.109069   
4            2.966534              2.919994                      1.323450   

   percent_native_american_female  percent_asian_male  percent_asian_female  \
0                        0.268640            0.218840              0.220066   
1                        0.200267            0.056918              0.123358   
2                        0.159070            0.771497              0.835502   
3                        0.110902            0.118581              0.158537   
4                        1.397792            1.963412              2.189571   

   percent_hispanic_male  percent_hispanic_female  cumulative_confirmed_cases  \
0               2.688941                 2.099242                      298307   
1               1.003296                 0.753020                      166637   
2               2.194774                 1.780878                     8499646   
3               1.140316                 1.096268                      310992   
4              15.307941                15.182164                    59578494   

   cumulative_deaths  
0               4787  
1               3646  
2             145397  
3               6604  
4            1094167  

[5 rows x 34 columns]

## Transform Data

In [5]:
# Add 2016 bellwether code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] >= 5:
        l.append(1)
    elif data.iloc[i, 3] < 5:
        l.append(0)
data['bellwether_2016'] = np.array(l, dtype=int)

In [6]:
# Add 2020 bellwether code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] == 6:
        l.append(1)
    elif data.iloc[i, 3] < 6:
        l.append(0)
data['bellwether_2020'] = np.array(l, dtype=int)

In [9]:
# Add >4 wins code

l = []
for i, v in enumerate(data['wins']):
    if data.iloc[i, 3] >= 4:
        l.append(1)
    elif data.iloc[i, 3] < 4:
        l.append(0)
data['bellwether'] = np.array(l, dtype=int)

# General Information on Dataset

In [10]:
data.shape

(590, 37)

In [11]:
data.columns

Index(['fips', 'state', 'county', 'wins', 'poverty_rate', 'population',
       'persons_in_poverty', 'rural_urban_code', 'urban_influence_code',
       'metro', 'civilian_labor_force', 'unemployment_rate', 'employed',
       'unemployed', 'no_diploma', 'diploma_only',
       'some_college/associate_degree', 'completed_college/bachelor_degree',
       'percent_no_diploma', 'percent_diploma_only',
       'percent_some_college/associate_degree',
       'percent_completed_college/bachelor_degree', 'percent_white_male',
       'percent_white_female', 'percent_black_male', 'percent_black_female',
       'percent_native_american_male', 'percent_native_american_female',
       'percent_asian_male', 'percent_asian_female', 'percent_hispanic_male',
       'percent_hispanic_female', 'cumulative_confirmed_cases',
       'cumulative_deaths', 'bellwether_2016', 'bellwether_2020',
       'bellwether'],
      dtype='object')

In [12]:
X = data.drop(['fips', 'state', 'county', 'wins', 'bellwether_2016', 'bellwether_2020', 'bellwether'], axis=1)
X.columns

Index(['poverty_rate', 'population', 'persons_in_poverty', 'rural_urban_code',
       'urban_influence_code', 'metro', 'civilian_labor_force',
       'unemployment_rate', 'employed', 'unemployed', 'no_diploma',
       'diploma_only', 'some_college/associate_degree',
       'completed_college/bachelor_degree', 'percent_no_diploma',
       'percent_diploma_only', 'percent_some_college/associate_degree',
       'percent_completed_college/bachelor_degree', 'percent_white_male',
       'percent_white_female', 'percent_black_male', 'percent_black_female',
       'percent_native_american_male', 'percent_native_american_female',
       'percent_asian_male', 'percent_asian_female', 'percent_hispanic_male',
       'percent_hispanic_female', 'cumulative_confirmed_cases',
       'cumulative_deaths'],
      dtype='object')

In [14]:
X2=X[["poverty_rate", "population", "unemployment_rate", "percent_diploma_only", "percent_some_college/associate_degree", "percent_white_female", "percent_white_male", "percent_black_male","percent_black_female", "cumulative_confirmed_cases"]]
#X2=X[["poverty_rate"]]

In [15]:
X2.shape
X=X2

In [16]:
y = data.bellwether

In [17]:
X.shape

(590, 10)

# Splitting into Train-Test Data

In [18]:
# split data into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=0)

In [19]:
# general info on data
y_train.shape

(442,)

In [20]:
# general info on data
X_train.shape

(442, 10)

In [21]:
# general info on data
X_test.shape

(148, 10)

In [22]:
# printing dictionary of how many bellwethers and nonbellwethers there are in the train set -- needs to be fixed with oversampling
print(Counter(y_train))

Counter({1: 277, 0: 165})


In [23]:
# printing dictionary of how many bellwethers and nonbellwethers there are in the test set
print(Counter(y_test))

Counter({1: 92, 0: 56})


In [ ]:
#nothing to see here

# Balancing the Test Set

In [42]:
# define oversampling strategy - 2:1 ratio of nonbellwethers to bellwethers
oversample = RandomOverSampler(sampling_strategy=1)
# fit and apply the transform
X_over_test, y_over_test = oversample.fit_resample(X_test, y_test)
# summarize class distribution
print(Counter(y_over_test))

Counter({1: 92, 0: 92})


#### Balacing the Train Set

In [43]:
# define oversampling strategy - 2:1 ratio of nonbellwethers to bellwethers
oversample = RandomOverSampler(sampling_strategy=1)
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(y_over))

Counter({1: 277, 0: 277})


In [283]:
model = RandomForestClassifier()

model.fit(X_over, y_over)

predicted_labels = model.predict(X_over_test)

#report=classification_report(y_over_test, predicted_labels)
#print(report)

metrics.confusion_matrix(y_over_test, predicted_labels)

#print (accuracy_score(y_over_test, predicted_labels))

array([[577, 123],
       [276,  74]])

In [44]:
#plt.figure(figsize=(25,30))
#plt.barh(X.columns, model.feature_importances_)
#print(model.feature_importances_)
X.columns

Index(['poverty_rate', 'population', 'unemployment_rate',
       'percent_diploma_only', 'percent_some_college/associate_degree',
       'percent_white_female', 'percent_white_male', 'percent_black_male',
       'percent_black_female', 'cumulative_confirmed_cases'],
      dtype='object')

In [45]:
X_over.columns

Index(['poverty_rate', 'population', 'unemployment_rate',
       'percent_diploma_only', 'percent_some_college/associate_degree',
       'percent_white_female', 'percent_white_male', 'percent_black_male',
       'percent_black_female', 'cumulative_confirmed_cases'],
      dtype='object')

In [46]:
X_over.shape

(554, 10)

# Tuning the Hyperparameters

In [47]:
d_range = np.arange(1, 20, 2)
n_range = np.arange(50, 500, 50)

In [48]:
training_error = []
testing_error = []

for d in d_range:
    for n in n_range:
        rf_model = RandomForestClassifier(n_estimators=n,max_depth=d, 
                                          random_state=0)

        rf_model.fit(X_over, y_over)

        y_pred_class = rf_model.predict(X_over)
        training_accuracy = metrics.accuracy_score(y_over, y_pred_class)
        training_error.append(1 - training_accuracy)

        y_pred_class = rf_model.predict(X_over_test)
        testing_accuracy = metrics.accuracy_score(y_over_test, y_pred_class)
        testing_error.append(1 - testing_accuracy)

In [49]:
parameters_d = []
parameters_n = []

for d in d_range:
    for n in n_range:
        parameters_d.append(d)
        parameters_n.append(n)

In [50]:
# Create a DataFrame of the search with training error, and testing error.

column_dict = {'d': parameters_d, 'n': parameters_n, 'training error': training_error, 'testing error': testing_error}
df = pd.DataFrame(column_dict)
df.head()

d    n  training error  testing error
0  1   50        0.351986       0.347826
1  1  100        0.348375       0.342391
2  1  150        0.344765       0.342391
3  1  200        0.351986       0.342391
4  1  250        0.342960       0.342391

In [51]:
df = df.sort_values(by='testing error', ascending=True)
df.head()

d    n  training error  testing error
45  11   50        0.000000       0.277174
20   5  150        0.155235       0.277174
51  11  350        0.000000       0.282609
40   9  250        0.003610       0.293478
29   7  150        0.032491       0.293478

# Machine Learning Model Based on Adjusted Train-Test

In [53]:
attributes_importance={}
for i in X_over.columns:
    attributes_importance[i]=0

In [317]:
def remove_lowest_key():
    key=min(attributes_importance,key=attributes_importance.get)
    del attributes_importance[key]
    X=X.drop([key],axis=1)

In [54]:
#200 7
#9 450
model=RandomForestClassifier(n_estimators=150,max_depth=5)
accuracy_list=[]

for i in range(30):
    model.fit(X_over,y_over)
    count=0
    y_pred_class = model.predict(X_over_test)
    report=classification_report(y_over_test, y_pred_class,output_dict=True)
    precision=report["1"]["precision"]
    accuracy_list.append(precision)
    for i in X_over.columns:
        attributes_importance[i]+=model.feature_importances_[count]
        count+=1
for i in X_over.columns:
    attributes_importance[i]=attributes_importance[i]/30

# for i in range(15):
#     attributes_importance=dict.fromkeys(attributes_importance,0)
#     for i in range(30):
#         model.fit(X_over,y_over)
#         count=0
#         for i in X_over.columns:
#             attributes_importance[i]+=model.feature_importances_[count]
#             count+=1
#     for i in X_over.columns:
#         attributes_importance[i]=attributes_importance[i]/30
#     key=min(attributes_importance,key=attributes_importance.get)
#     del attributes_importance[key]
#     X_over=X_over.drop([key],axis=1)
#     key=min(attributes_importance,key=attributes_importance.get)
#     del attributes_importance[key]
#     X_over=X_over.drop([key],axis=1)

   
    
    

In [55]:
y_over_test.shape

(184,)

In [56]:
print(attributes_importance)

{'poverty_rate': 0.12795758797440523, 'population': 0.0867318208534232, 'unemployment_rate': 0.18481740799396054, 'percent_diploma_only': 0.16279143006623092, 'percent_some_college/associate_degree': 0.30278144008630825, 'percent_white_female': 0.0109444173727754, 'percent_white_male': 0.012138940920908808, 'percent_black_male': 0.010130839922307122, 'percent_black_female': 0.010197581541012626, 'cumulative_confirmed_cases': 0.09150853326866777}


In [249]:
sum=0
for i in X_over.columns:
    sum+=attributes_importance[i]
print(sum)

1.0


In [57]:
#average of accuracies from 30 runs of the model
statistics.mean(accuracy_list)

0.6739011787250909

In [58]:
#standard deviation of 30 accuracies
statistics.stdev(accuracy_list)

0.019250961248794485

# Other Bagging Attempt (for testing purposes)

In [88]:
#bagged decision trees from https://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 50
model = BaggingClassifier(base_estimator=cart, max_samples=200, max_features = 100, n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

0.9004484304932736


In [89]:
model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=100,
         max_samples=200, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=7, verbose=0, warm_start=False)

In [92]:
model.score(X_train,y_train)

0.9022421524663677

In [93]:
y_pred_class = model.predict(X_test)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
testing_error = 1 - testing_accuracy

In [94]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[660,   2],
       [ 82,   0]])

In [46]:
#bg = BaggingClassifier(DecisionTreeClassifier(), max_samples=2230, max_features = 165, n_estimators=5)
bg = BaggingClassifier()

In [47]:
bg.fit(X_train,y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [48]:
bg.score(X_test,y_test)

0.8817204301075269

In [49]:
bg.score(X_train,y_train)

0.9838565022421525

In [61]:
y_pred_class = bg.predict(X_train)
training_accuracy = metrics.accuracy_score(y_train, y_pred_class)
training_error = 1 - training_accuracy

y_pred_class = bg.predict(X_test)
testing_accuracy = metrics.accuracy_score(y_test, y_pred_class)
testing_error = 1 - testing_accuracy

print("Best Bagging Model Test Accuracy: {0}, Test Error: {1}".format(training_accuracy, training_error))
print("Best Bagging Model Test Accuracy: {0}, Test Error: {1}".format(testing_accuracy, testing_error))

Best Bagging Model Test Accuracy: 0.9838565022421525, Test Error: 0.016143497757847514
Best Bagging Model Test Accuracy: 0.8817204301075269, Test Error: 0.11827956989247312


In [62]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[656,   6],
       [ 82,   0]])

In [236]:
report=classification_report(y_over_test, y_pred_class)
print(report)

              precision    recall  f1-score   support

           0       0.68      1.00      0.80       700
           1       0.83      0.04      0.08       350

    accuracy                           0.68      1050
   macro avg       0.75      0.52      0.44      1050
weighted avg       0.73      0.68      0.56      1050

